In [444]:
# Housekeeping
import requests
import datetime
import pandas as pd
import numpy as np
import io
import os
import trueskill
import itertools
import math

import oeutils

pd.options.display.max_rows = 100
pd.options.display.max_columns = 500

workingdir = '../data/'
years = ['2018', '2019', '2020', '2021']

#oeutils.download(workingdir, years)
df = oeutils.read('../data/')
df = oeutils.clean(df, split_on='team', keep_identities=False, keep_leagues=False, keep_columns=False)

In [445]:
df.shape

(37246, 117)

In [446]:
df['league'].unique()

array(['KeSPA', 'LPL', 'LFL', 'LEC', 'LCS.A', 'CBLOL', 'LCS', 'UL', 'OPL',
       'VCS', 'TCL', 'TRA', 'BRCC', 'UKLC', 'UPL', 'LCK', 'CK', 'LJL',
       'LLA', 'LCL', 'PCS', 'EM', 'LDL', 'BIG', 'OCS', 'Riot', 'MSC',
       'RCL', 'CU', 'NLC', 'WCS', 'NEST', 'NASG', 'DC', 'EU LCS', 'LLN',
       'LMS', 'CLS', 'NA LCS', 'TCS', 'CIS CL', 'GPL', 'MSI'],
      dtype=object)

In [447]:
df.head()

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,playerid,side,position,player,team,champion,ban1,ban2,ban3,ban4,ban5,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,doublekills,triplekills,quadrakills,pentakills,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,dragons (type unknown),elders,opp_elders,firstherald,heralds,opp_heralds,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,gspd,total cs,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,opp_killsat10,opp_assistsat10,opp_deathsat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
10,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,100,Blue,team,NaN,SANDBOX Gaming,NaN,LeBlanc,Irelia,Rek'Sai,Yasuo,Renekton,2220,1,17,11,33,17,11,3.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.4595,0.7568,0.0,2.0,3.0,2.0,3.0,0.0,2.0,0.0,0.0,NaN,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,8.0,4.0,0.0,0.0,1.0,0.0,77679,2099.4324,NaN,1975.3243,2302.4595,191,5.1622,97,2.6216,67,380.0,10.2703,68927,44905,1213.6486,NaN,62275.0,0.004023,NaN,1049.0,216,166,0,34.1892,14615.0,17845.0,330.0,14498.0,17982.0,322.0,117.0,-137.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,21953.0,27219.0,507.0,22778.0,28884.0,519.0,-825.0,-1665.0,-12.0,0.0,0.0,1.0,1.0,2.0,0.0
11,ESPORTSTMNT03/1241318,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 07:33:26,1,9.24,200,Red,team,NaN,T1,NaN,Lucian,Akali,Lee Sin,Olaf,Jarvan IV,2220,0,11,17,25,11,17,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,0.2973,0.7568,1.0,3.0,2.0,3.0,2.0,1.0,1.0,1.0,0.0,NaN,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,4.0,8.0,1.0,1.0,0.0,1.0,48685,1315.8108,NaN,2934.9459,3135.6486,172,4.6486,91,2.4595,69,318.0,8.5946,62709,38687,1045.5946,NaN,62025.0,-0.004023,NaN,1021.0,271,193,5,34.9189,14498.0,17982.0,322.0,14615.0,17845.0,330.0,-117.0,137.0,-8.0,0.0,0.0,0.0,0.0,0.0,0.0,22778.0,28884.0,519.0,21953.0,27219.0,507.0,825.0,1665.0,12.0,1.0,2.0,0.0,0.0,0.0,1.0
22,ESPORTSTMNT03/1241322,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 09:00:58,2,9.24,100,Blue,team,NaN,T1,NaN,Syndra,LeBlanc,Rumble,Braum,Leona,2227,0,5,21,16,5,21,1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.1347,0.7005,0.0,1.0,4.0,1.0,4.0,0.0,0.0,0.0,1.0,NaN,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,9.0,1.0,0.0,0.0,1.0,72963,1965.7746,NaN,2324.1042,2729.5016,144,3.8797,95,2.5595,58,254.0,6.8433,57283,33190,894.2075,NaN,56575.0,-0.122771,NaN,1011.0,183,151,1,32.1688,14180.0,16947.0,300.0,16194.0,19096.0,334.0,-2014.0,-2149.0,-34.0,0.0,0.0,3.0,3.0,3.0,0.0,21714.0,26869.0,501.0,27198.0,31702.0,557.0,-5484.0,-4833.0,-56.0,0.0,0.0,6.0,6.0,7.0,0.0
23,ESPORTSTMNT03/1241322,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2020,NaN,0,2020-01-03 09:00:58,2,9.24,200,Red,team,NaN,SANDBOX Gaming,NaN,Lucian,Akali,Rek'Sai,Irelia,Camille,2227,1,21,5,41,21,5,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,0.5658,0.7005,1.0,4.0,1.0,4.0,1.0,0.0,1.0,1.0,2.0,NaN,0.0,0.0,1.0,2.0,0.0,1.0,2.0,0.0,1.0,9.0,2.0,0.0,1.0,1.0,0.0,65542,1765.8374,NaN,2861.2214,3192.7167,195,5.2537,90,2.4248,53,397.0,10.6960,72004,47911,1290.8217,NaN,63975.0,0.122771,NaN,976.0,270,126,48,33.5698,16194.0,19096.0,334.0,14180.0,16947.0,300.0,2014.0,2149.0,34.0,3.0,3.0,0.0,0.0,0.0,3.0,27198.0,31702.0,557.0,21714.0,26869.0,501.0,5484.0,4833.0,56.0,6.0,7.0,0.0,0.0,0.0,6.0
34

In [448]:
lcs = df[df['league'].isin(['LCS', 'NA LCS'])]
lcs.shape

(1480, 117)

In [449]:
lcs['team'].unique()

array(['Cloud9', 'Team Liquid', 'Counter Logic Gaming', 'Dignitas',
       '100 Thieves', 'Golden Guardians', 'Immortals', 'FlyQuest',
       'Evil Geniuses', 'TSM', 'OpTic Gaming', 'Clutch Gaming',
       'Echo Fox'], dtype=object)

In [450]:
lcs = lcs.sort_values(by=['date', 'team'])

In [451]:
team_map = {
    'OpTic Gaming': 'Immortals',
    'Echo Fox': 'Evil Geniuses',
    'Clutch Gaming':'Dignitas'
}

def replace_team_name(team_to_replace):
    if team_to_replace in team_map.keys():
        return team_map[team_to_replace]
    else:
        return team_to_replace

lcs['team'] = lcs.apply(lambda row: replace_team_name(row['team']), axis=1)
lcs['team'].unique()

array(['TSM', 'Team Liquid', '100 Thieves', 'Immortals', 'Dignitas',
       'Golden Guardians', 'Evil Geniuses', 'FlyQuest', 'Cloud9',
       'Counter Logic Gaming'], dtype=object)

In [452]:
lcs.head()

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,playerid,side,position,player,team,champion,ban1,ban2,ban3,ban4,ban5,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,doublekills,triplekills,quadrakills,pentakills,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,dragons (type unknown),elders,opp_elders,firstherald,heralds,opp_heralds,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,gspd,total cs,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,opp_killsat10,opp_assistsat10,opp_deathsat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
1703,TRLH1/1002440062,complete,http://matchhistory.na.leagueoflegends.com/en/...,NA LCS,2018,Spring,0,2018-01-20 20:50:09,1,8.01,200,Red,team,NaN,TSM,NaN,Kog'Maw,Tahm Kench,Ezreal,Braum,Shen,1687,0,1,11,2,1,11,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0356,0.4268,1.0,2.0,2.0,2.0,2.0,0.0,0.0,1.0,1.0,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,10.0,0.0,0.0,0.0,1.0,27569,980.5216,NaN,2336.5857,2197.7712,95,3.3788,38,1.3515,24,221.0,7.8601,47020,28435,1011.3219,NaN,44225.0,-0.160319,NaN,951.0,149,100,4,39.1227,15755.0,18703.0,374.0,15970.0,18541.0,341.0,-215.0,162.0,33.0,0.0,0.0,1.0,1.0,1.0,0.0,24118.0,29903.0,599.0,26505.0,31407.0,551.0,-2387.0,-1504.0,48.0,0.0,0.0,4.0,4.0,10.0,0.0
1702,TRLH1/1002440062,complete,http://matchhistory.na.leagueoflegends.com/en/...,NA LCS,2018,Spring,0,2018-01-20 20:50:09,1,8.01,100,Blue,team,NaN,Team Liquid,NaN,Ornn,Zoe,Azir,Thresh,Kha'Zix,1687,1,11,1,36,11,1,1.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,0.3912,0.4268,0.0,2.0,2.0,2.0,2.0,1.0,0.0,0.0,1.0,NaN,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,10.0,2.0,1.0,1.0,1.0,0.0,45826,1629.8518,NaN,1814.7955,2226.2952,114,4.0545,32,1.1381,36,224.0,7.9668,59283,40698,1447.4689,NaN,51933.0,0.160319,NaN,869.0,196,124,32,37.8779,15970.0,18541.0,341.0,15755.0,18703.0,374.0,215.0,-162.0,-33.0,1.0,1.0,0.0,0.0,0.0,1.0,26505.0,31407.0,551.0,24118.0,29903.0,599.0,2387.0,1504.0,-48.0,4.0,10.0,0.0,0.0,0.0,4.0
1714,TRLH1/1002440076,complete,http://matchhistory.na.leagueoflegends.com/en/...,NA LCS,2018,Spring,0,2018-01-20 22:38:20,1,8.01,100,Blue,team,NaN,100 Thieves,NaN,Tahm Kench,Orianna,Kalista,Taric,Ornn,3873,1,9,7,27,9,7,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,0.1394,0.2479,1.0,5.0,1.0,3.0,0.0,0.0,1.0,0.0,2.0,NaN,2.0,1.0,0.0,0.0,1.0,1.0,3.0,2.0,0.0,11.0,7.0,0.0,0.0,3.0,1.0,112531,1743.3153,NaN,3514.5778,6596.7622,245,3.7955,144,2.2308,37,609.0,9.4345,120533,79650,1233.9272,NaN,112358.0,0.098992,NaN,1938.0,320,199,38,34.9806,15137.0,19223.0,344.0,16248.0,19671.0,351.0,-1111.0,-448.0,-7.0,0.0,0.0,0.0,0.0,0.0,0.0,23061.0,30627.0,560.0,24670.0,31467.0,564.0,-1609.0,-840.0,-4.0,0.0,0.0,0.0,0.0,0.0,0.0
1715,TRLH1/1002440076,complete,http://matchhistory.na.leagueoflegends.com/en/...,NA LCS,2018,Spring,0,2018-01-20 22:38:20,1,8.01,200,Red,team,NaN,Immortals,NaN,Zoe,Evelynn,Azir,Gangplank,Gnar,3873,0,7,9,17,7,9,1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.1084,0.2479,0.0,1.0,5.0,0.0,3.0,0.0,0.0,0.0,0.0,NaN,1.0,2.0,1.0,1.0,0.0,0.0,2.0,3.0,1.0,7.0,11.0,1.0,1.0,1.0,3.0,187337,2902.1998,NaN,2460.2634,3162.9589,300,4.6476,101,1.5647,45,570.0,8.8304,114382,73499,1138.6367,NaN,101760.0,-0.098992,NaN,1718.0,362,238,29,32.2231,16248.0,19671.0,351.0,15137.0,19223.0,344.0,1111.0,448.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,24670.0,31467.0,564.0,23061.0,30627.0,560.0,1609.0,840.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
1726,

In [453]:
input_data = lcs[['gameid', 'date', 'playerid', 'side', 'team', 'result']]
single_gameid_input_data = input_data.drop_duplicates(subset=['gameid'])
input_data.head()

,gameid,date,playerid,side,team,result
1703,TRLH1/1002440062,2018-01-20 20:50:09,200,Red,TSM,0
1702,TRLH1/1002440062,2018-01-20 20:50:09,100,Blue,Team Liquid,1
1714,TRLH1/1002440076,2018-01-20 22:38:20,100,Blue,100 Thieves,1
1715,TRLH1/1002440076,2018-01-20 22:38:20,200,Red,Immortals,0
1726,TRLH1/1002440084,2018-01-21 00:14:20,100,Blue,Dignitas,1


In [454]:
columns = ['gameid', 'date', \
           'blue_team_name', 'blue_team_result', \
           'red_team_name']

lcs_rating = pd.DataFrame(columns=columns)

## Trueskill

In [455]:
ts = trueskill.TrueSkill(draw_probability=0.0)

In [456]:
team_ratings_dict = dict()
for i in input_data['team'].unique():
    team_ratings_dict[i] = (ts.create_rating(),)
    
print(team_ratings_dict)

{'TSM': (trueskill.Rating(mu=25.000, sigma=8.333),), 'Team Liquid': (trueskill.Rating(mu=25.000, sigma=8.333),), '100 Thieves': (trueskill.Rating(mu=25.000, sigma=8.333),), 'Immortals': (trueskill.Rating(mu=25.000, sigma=8.333),), 'Dignitas': (trueskill.Rating(mu=25.000, sigma=8.333),), 'Golden Guardians': (trueskill.Rating(mu=25.000, sigma=8.333),), 'Evil Geniuses': (trueskill.Rating(mu=25.000, sigma=8.333),), 'FlyQuest': (trueskill.Rating(mu=25.000, sigma=8.333),), 'Cloud9': (trueskill.Rating(mu=25.000, sigma=8.333),), 'Counter Logic Gaming': (trueskill.Rating(mu=25.000, sigma=8.333),)}


In [457]:
### Lambda Apply w Opp as Key
def setup_match(df, gameid):
    blue = df.query(f'(gameid == "{gameid}") and (side == "Blue")')
    date = blue['date'].iloc[0]
    blue_team = blue['team'].iloc[0]
    blue_result = blue['result'].iloc[0]
    red = df.query(f'(gameid == "{gameid}") and (side == "Red")')
    red_team = red['team'].iloc[0]
    match = pd.Series([gameid, date, blue_team, blue_result, red_team])
    return match

lcs_rating[['gameid', 'date', 'blue_team_name', 'blue_team_result', 'red_team_name']] = single_gameid_input_data.apply(lambda row: setup_match(input_data, row['gameid']),axis=1)


In [458]:
lcs_rating.shape


(740, 5)

In [459]:
lcs_rating = lcs_rating.reset_index(drop=True)
lcs_rating = lcs_rating.drop_duplicates(subset=['gameid'])
lcs_rating = lcs_rating.sort_values(by=['date'])

In [461]:
lcs_rating.head(20)

,gameid,date,blue_team_name,blue_team_result,red_team_name
0,TRLH1/1002440062,2018-01-20 20:50:09,Team Liquid,1,TSM
1,TRLH1/1002440076,2018-01-20 22:38:20,100 Thieves,1,Immortals
2,TRLH1/1002440084,2018-01-21 00:14:20,Dignitas,1,Golden Guardians
3,TRLH1/1002440095,2018-01-21 01:20:39,Evil Geniuses,1,FlyQuest
4,TRLH1/1002440106,2018-01-21 02:16:56,Counter Logic Gaming,0,Cloud9
5,TRLH1/1002440127,2018-01-21 21:49:32,Immortals,0,Team Liquid
6,TRLH1/1002440132,2018-01-21 22:46:16,FlyQuest,1,TSM
7,TRLH1/1002440143,2018-01-22 00:00:28,100 Thieves,1,Counter Logic Gaming
8,TRLH1/1002440150,2018-01-22 01:00:51,Golden Guardians,0,Cloud9
9,TRLH1/1002440161,2018-01-22 02:02:56,Evil Geniuses,1,Dignitas


In [462]:
analyzed_gameids = {}

def win_probability(team1, team2, trueskill_global_env):
    delta_mu = sum(r.mu for r in team1) - sum(r.mu for r in team2)
    sum_sigma = sum(r.sigma ** 2 for r in itertools.chain(team1, team2))
    size = len(team1) + len(team2)
    denom = math.sqrt(size * (trueskill_global_env.beta ** 2 ) + sum_sigma)
    return trueskill_global_env.cdf(delta_mu / denom)

def update_trueskill(rating_dict, gameid_dict, blue_team, blue_team_result, red_team, gameid):
    rating_groups = [rating_dict[blue_team], rating_dict[red_team]]
    rating_dict[blue_team][0].mu
    blue_mu = rating_dict[blue_team][0].mu
    blue_sigma = rating_dict[blue_team][0].sigma
    red_mu = rating_dict[red_team][0].mu
    red_sigma = rating_dict[red_team][0].sigma
    blue_team_win_prob = win_probability(rating_dict[blue_team], rating_dict[red_team], ts)
    print(blue_team, blue_mu, '---', red_team, red_mu)
    # Update ratings
    if blue_team_result == 1:
        rated_rating_groups = ts.rate(rating_groups, ranks=[0, 1]) # for ranks 0 is winner 
    else:
        rated_rating_groups = ts.rate(rating_groups, ranks=[1, 0])
    # Return values for new columns
    ts_update = pd.Series([blue_team_win_prob, blue_mu, blue_sigma, red_mu, red_sigma])
    print(ts_update)
    # update the rating dictionary
    print(rating_dict)
    rating_dict[blue_team] = rated_rating_groups[0]
    rating_dict[red_team] = rated_rating_groups[1]
    print(rating_dict)
    if gameid in gameid_dict:
        return gameid_dict[gameid]
    else:
        gameid_dict[gameid] = ts_update
        return ts_update

#lcs_rating.apply(lambda row: update_trueskill(rating_dict, row['blue_team_name'], row['blue_team_result'], row['red_team_name']), axis=1)
lcs_rating[['blue_team_win_prob', 'blue_mu', 'blue_sigma', 'red_mu', 'red_sigma']] = lcs_rating.apply(lambda row: update_trueskill(team_ratings_dict, analyzed_gameids, row['blue_team_name'], row['blue_team_result'], row['red_team_name'], row['gameid']), axis=1)

Team Liquid 25.0 --- TSM 25.0
0     0.500000
1    25.000000
2     8.333333
3    25.000000
4     8.333333
dtype: float64
{'TSM': (trueskill.Rating(mu=25.000, sigma=8.333),), 'Team Liquid': (trueskill.Rating(mu=25.000, sigma=8.333),), '100 Thieves': (trueskill.Rating(mu=25.000, sigma=8.333),), 'Immortals': (trueskill.Rating(mu=25.000, sigma=8.333),), 'Dignitas': (trueskill.Rating(mu=25.000, sigma=8.333),), 'Golden Guardians': (trueskill.Rating(mu=25.000, sigma=8.333),), 'Evil Geniuses': (trueskill.Rating(mu=25.000, sigma=8.333),), 'FlyQuest': (trueskill.Rating(mu=25.000, sigma=8.333),), 'Cloud9': (trueskill.Rating(mu=25.000, sigma=8.333),), 'Counter Logic Gaming': (trueskill.Rating(mu=25.000, sigma=8.333),)}
{'TSM': (trueskill.Rating(mu=20.795, sigma=7.195),), 'Team Liquid': (trueskill.Rating(mu=29.205, sigma=7.195),), '100 Thieves': (trueskill.Rating(mu=25.000, sigma=8.333),), 'Immortals': (trueskill.Rating(mu=25.000, sigma=8.333),), 'Dignitas': (trueskill.Rating(mu=25.000, sigma=8.333)

Cloud9 24.54575278355947 --- Immortals 21.589452700574473
0     0.673007
1    24.545753
2     1.883888
3    21.589453
4     2.286723
dtype: float64
{'TSM': (trueskill.Rating(mu=28.063, sigma=1.794),), 'Team Liquid': (trueskill.Rating(mu=29.094, sigma=1.571),), '100 Thieves': (trueskill.Rating(mu=26.717, sigma=1.609),), 'Immortals': (trueskill.Rating(mu=21.589, sigma=2.287),), 'Dignitas': (trueskill.Rating(mu=26.715, sigma=1.550),), 'Golden Guardians': (trueskill.Rating(mu=21.271, sigma=2.118),), 'Evil Geniuses': (trueskill.Rating(mu=27.450, sigma=1.753),), 'FlyQuest': (trueskill.Rating(mu=21.328, sigma=2.198),), 'Cloud9': (trueskill.Rating(mu=24.546, sigma=1.884),), 'Counter Logic Gaming': (trueskill.Rating(mu=23.721, sigma=2.009),)}
{'TSM': (trueskill.Rating(mu=28.063, sigma=1.794),), 'Team Liquid': (trueskill.Rating(mu=29.094, sigma=1.571),), '100 Thieves': (trueskill.Rating(mu=26.717, sigma=1.609),), 'Immortals': (trueskill.Rating(mu=21.164, sigma=2.214),), 'Dignitas': (trueskill.Ra

dtype: float64
{'TSM': (trueskill.Rating(mu=26.986, sigma=1.114),), 'Team Liquid': (trueskill.Rating(mu=29.436, sigma=1.185),), '100 Thieves': (trueskill.Rating(mu=26.800, sigma=1.114),), 'Immortals': (trueskill.Rating(mu=24.097, sigma=1.478),), 'Dignitas': (trueskill.Rating(mu=24.821, sigma=1.238),), 'Golden Guardians': (trueskill.Rating(mu=21.288, sigma=1.495),), 'Evil Geniuses': (trueskill.Rating(mu=26.214, sigma=1.229),), 'FlyQuest': (trueskill.Rating(mu=23.765, sigma=1.386),), 'Cloud9': (trueskill.Rating(mu=25.918, sigma=1.225),), 'Counter Logic Gaming': (trueskill.Rating(mu=23.048, sigma=1.416),)}
{'TSM': (trueskill.Rating(mu=26.986, sigma=1.114),), 'Team Liquid': (trueskill.Rating(mu=29.436, sigma=1.185),), '100 Thieves': (trueskill.Rating(mu=26.800, sigma=1.114),), 'Immortals': (trueskill.Rating(mu=24.097, sigma=1.478),), 'Dignitas': (trueskill.Rating(mu=24.821, sigma=1.238),), 'Golden Guardians': (trueskill.Rating(mu=21.085, sigma=1.474),), 'Evil Geniuses': (trueskill.Rating(m

Cloud9 27.25871945741346 --- Evil Geniuses 25.501785623327454
0     0.613923
1    27.258719
2     1.031664
3    25.501786
4     1.014702
dtype: float64
{'TSM': (trueskill.Rating(mu=27.637, sigma=0.938),), 'Team Liquid': (trueskill.Rating(mu=29.449, sigma=1.015),), '100 Thieves': (trueskill.Rating(mu=24.785, sigma=0.988),), 'Immortals': (trueskill.Rating(mu=24.086, sigma=1.173),), 'Dignitas': (trueskill.Rating(mu=23.586, sigma=1.058),), 'Golden Guardians': (trueskill.Rating(mu=23.365, sigma=1.119),), 'Evil Geniuses': (trueskill.Rating(mu=25.502, sigma=1.015),), 'FlyQuest': (trueskill.Rating(mu=24.078, sigma=1.056),), 'Cloud9': (trueskill.Rating(mu=27.259, sigma=1.032),), 'Counter Logic Gaming': (trueskill.Rating(mu=23.527, sigma=1.151),)}
{'TSM': (trueskill.Rating(mu=27.637, sigma=0.938),), 'Team Liquid': (trueskill.Rating(mu=29.449, sigma=1.015),), '100 Thieves': (trueskill.Rating(mu=24.785, sigma=0.988),), 'Immortals': (trueskill.Rating(mu=24.086, sigma=1.173),), 'Dignitas': (trueskil

dtype: float64
{'TSM': (trueskill.Rating(mu=26.679, sigma=0.881),), 'Team Liquid': (trueskill.Rating(mu=29.267, sigma=0.926),), '100 Thieves': (trueskill.Rating(mu=24.800, sigma=0.916),), 'Immortals': (trueskill.Rating(mu=24.020, sigma=1.005),), 'Dignitas': (trueskill.Rating(mu=25.037, sigma=0.916),), 'Golden Guardians': (trueskill.Rating(mu=23.133, sigma=1.002),), 'Evil Geniuses': (trueskill.Rating(mu=24.326, sigma=0.935),), 'FlyQuest': (trueskill.Rating(mu=23.954, sigma=0.961),), 'Cloud9': (trueskill.Rating(mu=27.670, sigma=0.916),), 'Counter Logic Gaming': (trueskill.Rating(mu=24.785, sigma=0.955),)}
{'TSM': (trueskill.Rating(mu=26.679, sigma=0.881),), 'Team Liquid': (trueskill.Rating(mu=29.267, sigma=0.926),), '100 Thieves': (trueskill.Rating(mu=24.912, sigma=0.913),), 'Immortals': (trueskill.Rating(mu=24.020, sigma=1.005),), 'Dignitas': (trueskill.Rating(mu=25.037, sigma=0.916),), 'Golden Guardians': (trueskill.Rating(mu=23.133, sigma=1.002),), 'Evil Geniuses': (trueskill.Rating(m

TSM 26.251016336603556 --- Evil Geniuses 25.054083290032576
0     0.578879
1    26.251016
2     0.838038
3    25.054083
4     0.861766
dtype: float64
{'TSM': (trueskill.Rating(mu=26.251, sigma=0.838),), 'Team Liquid': (trueskill.Rating(mu=27.875, sigma=0.873),), '100 Thieves': (trueskill.Rating(mu=24.657, sigma=0.860),), 'Immortals': (trueskill.Rating(mu=23.631, sigma=0.918),), 'Dignitas': (trueskill.Rating(mu=24.429, sigma=0.867),), 'Golden Guardians': (trueskill.Rating(mu=23.405, sigma=0.914),), 'Evil Geniuses': (trueskill.Rating(mu=25.054, sigma=0.862),), 'FlyQuest': (trueskill.Rating(mu=25.073, sigma=0.862),), 'Cloud9': (trueskill.Rating(mu=29.202, sigma=0.879),), 'Counter Logic Gaming': (trueskill.Rating(mu=24.168, sigma=0.889),)}
{'TSM': (trueskill.Rating(mu=26.331, sigma=0.837),), 'Team Liquid': (trueskill.Rating(mu=27.875, sigma=0.873),), '100 Thieves': (trueskill.Rating(mu=24.657, sigma=0.860),), 'Immortals': (trueskill.Rating(mu=23.631, sigma=0.918),), 'Dignitas': (trueskill.

{'TSM': (trueskill.Rating(mu=26.952, sigma=0.811),), 'Team Liquid': (trueskill.Rating(mu=28.198, sigma=0.847),), '100 Thieves': (trueskill.Rating(mu=24.508, sigma=0.837),), 'Immortals': (trueskill.Rating(mu=23.404, sigma=0.885),), 'Dignitas': (trueskill.Rating(mu=24.141, sigma=0.844),), 'Golden Guardians': (trueskill.Rating(mu=24.146, sigma=0.862),), 'Evil Geniuses': (trueskill.Rating(mu=24.907, sigma=0.833),), 'FlyQuest': (trueskill.Rating(mu=25.817, sigma=0.824),), 'Cloud9': (trueskill.Rating(mu=28.362, sigma=0.847),), 'Counter Logic Gaming': (trueskill.Rating(mu=23.231, sigma=0.870),)}
{'TSM': (trueskill.Rating(mu=26.952, sigma=0.811),), 'Team Liquid': (trueskill.Rating(mu=28.198, sigma=0.847),), '100 Thieves': (trueskill.Rating(mu=24.398, sigma=0.835),), 'Immortals': (trueskill.Rating(mu=23.404, sigma=0.885),), 'Dignitas': (trueskill.Rating(mu=24.141, sigma=0.844),), 'Golden Guardians': (trueskill.Rating(mu=24.146, sigma=0.862),), 'Evil Geniuses': (trueskill.Rating(mu=24.907, sigma

In [465]:
lcs_rating.shape

(740, 10)

In [466]:
lcs_rating.head(20)

,gameid,date,blue_team_name,blue_team_result,red_team_name,blue_team_win_prob,blue_mu,blue_sigma,red_mu,red_sigma
0,TRLH1/1002440062,2018-01-20 20:50:09,Team Liquid,1,TSM,0.500000,25.000000,8.333333,25.000000,8.333333
1,TRLH1/1002440076,2018-01-20 22:38:20,100 Thieves,1,Immortals,0.500000,25.000000,8.333333,25.000000,8.333333
2,TRLH1/1002440084,2018-01-21 00:14:20,Dignitas,1,Golden Guardians,0.500000,25.000000,8.333333,25.000000,8.333333
3,TRLH1/1002440095,2018-01-21 01:20:39,Evil Geniuses,1,FlyQuest,0.500000,25.000000,8.333333,25.000000,8.333333
4,TRLH1/1002440106,2018-01-21 02:16:56,Counter Logic Gaming,0,Cloud9,0.500000,25.000000,8.333333,25.000000,8.333333
5,TRLH1/1002440127,2018-01-21 21:49:32,Immortals,0,Team Liquid,0.185151,20.794527,7.194817,30.988437,6.555772
6,TRLH1/1002440132,2018-01-21 22:46:16,FlyQuest,1,TSM,0.562259,20.794527,7.194817,19.011563,6.555772
7,TRLH1/1002440143,2018-01-22 00:00:28,100 Thieves,1,Counter Logic Gaming,0.762798,29.205473,7.194817,20.794527,7.194817
8,TRLH1/1002440150,2018-01-22 01:00:51,Golden Guardians,0,Cloud9,0.237202,20.794527,7.194817,29.205473,7.194817
9,TRLH1/1002440161,2018-01-22 02:02:56,Evil Geniuses,1,Dignitas,0.500000,29.205473,7.194817,29.205473,7.194817


In [467]:
#print(analyzed_gameids)

In [469]:
lcs_rating.to_csv('./lcs_team_trueskill_2018-2020.csv', index=False)